In [1]:
import numpy as np
from scipy.stats import norm, binom

# 1. MLE (Early Strong Fusion Model)

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [35]:
def negLL_MLE(params, data):
    mu_a_tilde = np.array([1,2]) - params[0]
    mu_v_tilde = np.array([1,2]) - params[1]
    print(f"mu_a_tilde {mu_a_tilde}")
    print(f"mu_v_tilde {mu_v_tilde}")
    print(f"c_a {params[0]}")
    sigma_a = np.exp(params[2])
    sigma_v = np.exp(params[3])
    print(sigma_a)
    print(sigma_v)
    p_est = np.zeros((4, 2))

    p_est[0, :] = norm.cdf(mu_a_tilde / sigma_a)
    p_est[1, :] = norm.cdf(mu_v_tilde / sigma_v)
    w_a = sigma_v**2 / (sigma_a**2 + sigma_v**2)
    sigma_av = np.sqrt((sigma_a**2 * sigma_v**2) / (sigma_a**2 + sigma_v**2))

    for a in range(2):
        for v in range(2):
            mu_av_tilde = w_a * mu_a_tilde[a] + (1 - w_a) * mu_v_tilde[v]
            p_est[v + 2, a] = norm.cdf(mu_av_tilde / sigma_av)
    print(p_est)
    NegLL = 0
    for r in range(4):
        for c in range(2):
            NegLL -= np.log(binom.pmf(data[r, c], 20, p_est[r, c]))

    return NegLL

In [26]:
data = np.array([[1, 20], [6, 12], [6, 18], [5, 20]])
Nparams = 4
params0 = np.random.rand(Nparams) - 0.5

options = {'maxiter': 1e5, 'disp': False}

result = minimize(negLL_MLE, params0, args=(data,), options=options)

params = result.x
c_a = params[0]
c_v = params[1]
sigma_a = np.exp(params[2])
sigma_v = np.exp(params[3])
NegLL = result.fun

print(f"c_a: {c_a}")
print(f"c_v: {c_v}")
print(f"sigma_a: {sigma_a}")
print(f"sigma_v: {sigma_v}")
print(f"NegLL: {NegLL}")


C:\Users\Usuario\AppData\Local\Temp\ipykernel_10820\3103269676.py:24: RuntimeWarning: divide by zero encountered in log
  NegLL -= np.log(binom.pmf(data[r, c], 20, p_est[r, c]))
c:\Users\Usuario\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10820\3103269676.py:24: RuntimeWarning: divide by zero encountered in log
  NegLL -= np.log(binom.pmf(data[r, c], 20, p_est[r, c]))
c:\Users\Usuario\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10820\3103269676.py:24: RuntimeWarning: divide by zero encountered in log
  NegLL -= np.log(binom.pmf(data[r, c], 20, p_est[r, c]))


c_a: 1.3067386537825676
c_v: 1.6048261189683979
sigma_a: 0.35982653362444694
sigma_v: 1.1941965883702195
NegLL: 13.397682943464599


In [36]:
NLL=negLL_MLE(params, data)
print(NLL)

mu_a_tilde [-0.30673865  0.69326135]
mu_v_tilde [-0.60482612  0.39517388]
c_a 1.3067386537825676
0.35982653362444694
1.1941965883702195
[[0.1969787  0.97298864]
 [0.30626297 0.62964449]
 [0.16794121 0.95530413]
 [0.23553252 0.97382235]]
13.397682943464599


# 2. Probability matching model

In [3]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import binom

def negLL_FLMP(params, data):
    pa = np.exp(params[0:2]) / (np.exp(params[0:2]) + 1)
    pv = np.exp(params[2:4]) / (np.exp(params[2:4]) + 1)

    pav = np.zeros((2, 2))

    for a in range(2):
        for v in range(2):
            pav[v, a] = pa[a] * pv[v] / (pa[a] * pv[v] + (1 - pa[a]) * (1 - pv[v]))

    p_est = np.vstack([pa, pv, pav])

    NegLL = 0
    for r in range(4):
        for c in range(2):
            NegLL -= np.log(binom.pmf(data[r, c], 20, p_est[r, c]))

    return NegLL

data = np.array([[1, 20], [6, 12], [6, 18], [5, 20]])
Nparams = 4
params0 = np.random.rand(Nparams) - 0.5

options = {'maxiter': 1e5, 'disp': False}

result = minimize(negLL_FLMP, params0, args=(data,), options=options)

params = result.x
pa = np.exp(params[0:2]) / (np.exp(params[0:2]) + 1)
pv = np.exp(params[2:4]) / (np.exp(params[2:4]) + 1)
NegLL = result.fun

print(f"pa: {pa}")
print(f"pv: {pv}")
print(f"NegLL: {NegLL}")
print(f"Params{params}")


pa: [0.19264555 0.96777293]
pv: [0.40613869 0.60427978]
NegLL: 14.35414112005347
Params[-1.43291079  3.40219066 -0.37995112  0.42332985]
